# 代理基准测试：搜索 + 计算器

在这里，我们将介绍如何在代理可以访问计算器和搜索工具的任务上进行基准测试。

强烈建议您在启用跟踪的情况下进行任何评估/基准测试。请参阅[此处](https://langchain.readthedocs.io/en/latest/tracing.html)了解跟踪是什么以及如何设置。


In [ ]:
# Comment this out if you are NOT using tracing
import os
os.environ["LANGCHAIN_HANDLER"] = "langchain"

## 加载数据
首先，让我们加载数据。

In [ ]:
from langchain.evaluation.loading import load_dataset
dataset = load_dataset("agent-search-calculator")

## 设置链条
现在，我们需要加载一个能够回答这些问题的代理。

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import LLMMathChain
from langchain.agents import initialize_agent, Tool, load_tools
from langchain.agents import AgentType

tools = load_tools(['serpapi', 'llm-math'], llm=OpenAI(temperature=0))
agent = initialize_agent(tools, OpenAI(temperature=0), agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

## 进行预测

首先，我们可以逐个数据点进行预测。以这种细粒度的方式进行预测允许我们详细探索输出，并且还比同时运行多个数据点更便宜。

In [ ]:
print(dataset[0]['question'])
agent.run(dataset[0]['question'])

## 进行多个预测
现在，我们可以进行多个预测。

In [ ]:
agent.run(dataset[4]['question'])

In [ ]:
predictions = []
predicted_dataset = []
error_dataset = []
for data in dataset:
    new_data = {"input": data["question"], "answer": data["answer"]}
    try:
        predictions.append(agent(new_data))
        predicted_dataset.append(new_data)
    except Exception as e:
        predictions.append({"output": str(e), **new_data})
        error_dataset.append(new_data)

## 评估性能
现在，我们可以评估预测结果。首先，我们可以通过肉眼观察它们。

In [ ]:
predictions[0]

接下来，我们可以使用语言模型对它们进行程序化评分。

In [ ]:
from langchain.evaluation.qa import QAEvalChain

In [ ]:
llm = OpenAI(temperature=0)
eval_chain = QAEvalChain.from_llm(llm)
graded_outputs = eval_chain.evaluate(dataset, predictions, question_key="question", prediction_key="output")

我们可以将评分输出添加到“预测”字典中，然后得到等级的数量。

In [ ]:
for i, prediction in enumerate(predictions):
    prediction['grade'] = graded_outputs[i]['text']

In [ ]:
from collections import Counter
Counter([pred['grade'] for pred in predictions])

我们还可以过滤出错误的示例数据点并查看它们。

In [ ]:
incorrect = [pred for pred in predictions if pred['grade'] == " INCORRECT"]

In [ ]:
incorrect